## Exploring Malaria Simulation Output Data

In [ ]:
from masim_analysis.analysis import *

strategies = [
    "AL5",
    "AL4",
    "AL25-ASAQ75",
    "AL25-DHAPPQ75",
    "AL50-ASAQ50",
    "AL50-DHAPPQ50",
    "AL75-ASAQ25",
    "AL75-DHAPPQ25",
    "ASAQ",
    "ASAQ25-DHAPPQ75",
    "ASAQ50-DHAPPQ50",
    "ASAQ75-DHAPPQ25",
    "DHA-PPQ",
    "status_quo",
    "DHA-PPQ_3yrs_then_5day_AL50-ASAQ50",
    "DHA-PPQ_3yrs_then_AL50-DHAPPQ50",
]

In [ ]:
import os
import glob
#DATA_ROOT: str = os.path.join("G:\\", "My Drive")
DATA_ROOT: str = os.path.join("/", "home", "james", "Code", "Temple-Malaria-Simulation-Analysis")
#DATA_SOURCE: str = os.path.join(DATA_ROOT, "Code", "output_old")
DATA_SOURCE: str = os.path.join(DATA_ROOT, "output2", "rwa")
filelist = glob.glob(os.path.join(DATA_SOURCE, "*.db"))

Prefilter the data to clear out runs that didn't complete

In [ ]:
import shutil

for file in filelist:
    months = get_table(file, 'monthlydata')
    try:
        last_month = months['id'].to_list()[-1]
    except IndexError:
        print(f"File: {file} is missing data")
        continue
    if last_month < 385:
        print(f"File: {file} is missing data for month {last_month}")
        continue
    # move the file to the correct location
    shutil.move(file, os.path.join("good", os.path.basename(file)))

In [ ]:
# Create treatment failure plots for all strategies
# Note: due to incomplete batch runs, some strategies may not have data or have incomplete data
# Incomplete data will result in a ValueError when plotting due to a dimension mismatch

DATA_SOURCE: str = os.path.join(DATA_ROOT, "good")

for strategy in strategies:
    try:
        data = aggregate_failure_rates(DATA_SOURCE, strategy)
    except TypeError as e:
        print(e)
        continue
    except FileNotFoundError as e:
        print(e)
        continue
    if data is None:
        print(f"No data found for {strategy} or there was an aggregation error")
        continue
    try:
        fig, ax = plot_strategy_treatment_failure(data, strategy.replace("_", " ").title())
    except ValueError as e:
        print(e)
        plt.close(fig)
        continue
    plt.savefig(f"{strategy}.png")
    plt.close(fig)

In [ ]:
# Create a violin plot for a given genotype based on the row in agg_fqy
fig, ax = plt.subplots()
locationid = 1
for i, strategy in enumerate(strategies):
    try:
        agg_fqy = aggregate_resistant_genome_frequencies(DATA_SOURCE, strategy, 'H', 325, locationid)
    except TypeError as e:
        print(e)
        continue
    except FileNotFoundError as e:
        print(e)
        continue
    except ValueError as e:
        print(e)
        continue
    except IndexError as e:
        print(e)
        continue
    try:
        ax.violinplot(agg_fqy, positions=[i], showmeans=True, orientation='horizontal')
    except Exception as e:
        print(e)
        print(f"Error plotting {strategy}")
        continue
title = "Genotype Frequencies"# for Location {locationid}"
if locationid > 0:
    title += f" for Location {locationid}"
ax.set_title(title)
ax.set_xlabel("Frequency")
ax.set_ylabel("Strategy")
ax.set_yticks(range(len(strategies)))
ax.set_yticklabels(strategies)
plt.savefig("genotype_frequencies_violins.png")
plt.show()

Next we want to create Figure 2 from the [zubpko23 paper](https://mol.ax/pdf/zupko23b.pdf). Rather, than compare treatments, let's just do the results for one treatment per plot. 

Plot the following:
1. Number of treamtment failures plus 90% confidence interval
2. Resistant genometypes plus 90% confidence interval
3. PfPR2-10 (twelve month smoothed malaria prevelance in children 2-10 years old)

Plots are over ten years from when the treatment was started.

In [ ]:
for strategy in strategies:
    try:
        fig = plot_combined_strategy_aggragated_results(DATA_SOURCE, strategy, 'H', 1)
    except Exception as e:
        print(e)
        continue
    plt.savefig(f"{strategy}_combined.png")
    plt.show()
    plt.close(fig)